# Data Engineering

Now that we've loaded our data, it's time to do some real data engineering to answer our original question.

At this stage, Iceberg fades into the background, but we're able to pick and choose query engines to perform the various steps - this is the true power of Iceberg

In [1]:
import sqlalchemy as sa
from utils import engine, catalog
import polars as pl
pl.Config.set_fmt_str_lengths(50)
pl.Config.set_thousands_separator(True)

polars.config.Config

We want a way of identifying a given property, so hashing the address related fields seems the easiest. We'll create a dimension table for those addresses, so we can move those rows out of our final data, without losing the ability to filter later. 
```{note} SQL Partitioning
Note that we're adding partitioning to our tables directly through SQL here using a WITH statement on the CREATE TABLE
```

In [2]:
dim_address_sql = """
CREATE OR REPLACE TABLE housing.dim_address 
    WITH ( partitioning = ARRAY['bucket(address_id, 10)'] )
    AS (
    SELECT DISTINCT to_hex(md5(cast(
        coalesce(paon, '') ||
        coalesce(saon, '') ||
        coalesce(street, '') ||
        coalesce(locality, '') ||
        coalesce(town, '') ||
        coalesce(district, '') ||
        coalesce(county, '') ||
        coalesce(postcode, '')
    as varbinary))) AS address_id,
      paon,
      saon,
      street,
      locality,
      town,
      district,
      county,
      postcode
FROM housing.staging_prices)
"""

As described in the data dictionary, the monthly files incluce a `record_status` column which indicates whether a given record is a new record or if it is deleting or updating an existing record. In moving from our staging table to our fact table, we clean our data to ensure we respect the record_status

In [3]:
fct_prices_sql = """
CREATE OR REPLACE TABLE housing.fct_house_prices
    WITH ( partitioning = ARRAY['year(date_of_transfer)'] ) AS (
        WITH ranked_records AS (
            SELECT *,
            ROW_NUMBER () OVER (PARTITION BY transaction_id ORDER BY month(date_of_transfer) DESC) AS rn
            FROM housing.staging_prices
    ),
    latest_records AS (
        SELECT *
        FROM ranked_records
        WHERE rn = 1
    ),
    with_address_id AS (
        SELECT to_hex(md5(cast (
                coalesce(paon, '') ||
                coalesce(saon, '') ||
                coalesce(street, '') ||
                coalesce(locality, '') ||
                coalesce(town, '') ||
                coalesce(district, '') ||
                coalesce(county, '') ||
                coalesce(postcode, '')
            as varbinary))) AS address_id,
                transaction_id,
                price,
                date_of_transfer,
                property_type,
                new_property,
                duration,
                ppd_category_type
        FROM latest_records
        WHERE record_status != 'D' and ppd_category_type = 'A'
    )
    SELECT *
    FROM with_address_id
    )
"""

In [4]:
with engine.begin() as conn:
    num_rows_dim_address = conn.execute(sa.text(dim_address_sql)).fetchone()[0]
    num_rows_fct_prices = conn.execute(sa.text(fct_prices_sql)).fetchone()[0]

print(f"Created dim_address with {num_rows_dim_address:,} rows")
print(f"Created fct_prices with {num_rows_fct_prices:,} rows")

Created dim_address with 8,309,212 rows
Created fct_prices with 8,552,420 rows


Now that the data is loaded, we can create a Pyiceberg reference to it

In [5]:
fct_house_prices_t = catalog.load_table("housing.fct_house_prices")

For a change of pace, let's use `polars` to write our profits calculation. Some things are easier to express in SQL and some are nice to be able to do in Polars. The choice is yours!

In [6]:
polars_result = (
    pl.scan_iceberg(fct_house_prices_t)
    .with_columns(
        pl.col("date_of_transfer").min().over(pl.col("address_id")).alias("first_day"),
        pl.col("date_of_transfer").max().over(pl.col("address_id")).alias("last_day"),
        pl.col("price")
        .sort_by("date_of_transfer")
        .first()
        .over(pl.col("address_id"))
        .alias("first_price"),
        pl.col("price")
        .sort_by("date_of_transfer")
        .last()
        .over(pl.col("address_id"))
        .alias("last_price"),
    )
    .with_columns(
        pl.col("last_day").sub(pl.col("first_day")).dt.total_days().alias("days_held"),
        pl.col("last_price").sub(pl.col("first_price")).alias("profit"),
    )
    .filter(pl.col("days_held") != 0)
    .select(
        pl.col("address_id"),
        pl.col("first_day"),
        pl.col("last_day"),
        pl.col("first_price"),
        pl.col("last_price"),
        pl.col("days_held"),
        pl.col("profit"),
    )
    .unique()
).collect()

polars_result

address_id,first_day,last_day,first_price,last_price,days_held,profit
str,date,date,i32,i32,i64,i32
"""8690E862152B6D4CB7FF9F3B03C5E115""",2022-03-04,2024-09-27,"45,500","95,000",938,"49,500"
"""71912386175398BDBB44EEB6E3A5D061""",2019-01-25,2024-10-24,"287,000","450,000","2,099","163,000"
"""3D3F56D57E4015316242649118A4AD87""",2015-01-20,2016-06-09,"195,000","234,000",506,"39,000"
"""8221CE778B883E899FD98C20FDACCD52""",2017-03-27,2023-08-24,"175,000","262,500","2,341","87,500"
"""9FD87016486D0205AB4710BDA759A454""",2017-03-28,2021-09-24,"290,000","320,000","1,641","30,000"
…,…,…,…,…,…,…
"""52DCBA83936A56406D789B804B8663DF""",2017-12-14,2019-07-19,"345,000","417,000",582,"72,000"
"""0F800ABA47FAFB8CB87DB7FC8D885C6A""",2017-11-13,2020-09-30,"124,995","145,000","1,052","20,005"
"""E3A284566D63FA6791C0F5BF35BE2420""",2018-10-26,2023-08-18,"227,500","300,000","1,757","72,500"


Let's store the results in a table for future reference - since `polars` is arrow-based, we can use it to define the schema as well if we don't care as much about the details of the resulting schema

In [7]:
profits_t = catalog.create_table_if_not_exists("housing.profits", schema=polars_result.to_arrow().schema)

In [8]:
profits_t.overwrite(polars_result.to_arrow())

/usr/local/lib/python3.12/site-packages/pyiceberg/table/__init__.py:686: UserWarning: Delete operation did not match any records
  warnings.warn("Delete operation did not match any records")


To round out the selection of query engines, we can use `daft` to query our newly created table and calculate the mean profits for a given year

In [9]:
import daft

(
    daft.read_iceberg(profits_t)
    .groupby(daft.col("first_day").dt.year().alias("year"))
    .agg(daft.col("profit").mean())
    .sort(daft.col("year"))
    .collect(10)
)

yearInt32,profitFloat64
2015,72301.1885172374
2016,59703.10476277467
2017,57739.08672772205
2018,58670.54759435653
2019,64499.84229153804
2020,68087.81635511974
2021,57104.07844168653
2022,33779.063714804724
2023,45438.82723962127
2024,50560.72284122563
